<a href="https://colab.research.google.com/github/rcosilva/azami/blob/master/C%C3%B3digo_Completo_(Densidade%2C_Custos_Negativos_e_Caminhos_Elementares)_Pulse%2C_A_%2C_SSR_Inspired_e_AdvancedPulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import heapq
import time
import random

# Definindo infinito para custos
INF = float('inf')

# ==============================================================================
# CLASSE PULSO (para Algoritmo Pulse e AdvancedPulse)
# ==============================================================================
class Pulso:
    def __init__(self, no_atual, custo, recursos_consumidos, caminho, num_arestas=0):
        self.no_atual = no_atual
        self.custo = custo
        self.recursos_consumidos = list(recursos_consumidos)
        self.caminho = list(caminho) # Lista de nós no caminho
        self.num_arestas = num_arestas

    def __lt__(self, other):
        if self.custo != other.custo:
            return self.custo < other.custo
        if self.num_arestas != other.num_arestas:
            return self.num_arestas < other.num_arestas
        return len(self.caminho) < len(other.caminho)

# ==============================================================================
# SOLUCIONADOR RCSPP COM ALGORITMO PULSE (Caminhos Elementares)
# ==============================================================================
class SolucionadorRCSPPPulse:
    def __init__(self, num_nos, arestas, limites_recursos):
        self.num_nos = num_nos
        self.adj = [[] for _ in range(num_nos)]
        self.contem_custos_negativos = False
        for u, v, custo, consumos_recurso in arestas:
            if not (0 <= u < num_nos and 0 <= v < num_nos):
                raise ValueError(f"Nó inválido na aresta ({u}, {v}). Nós devem estar entre 0 e {num_nos-1}")
            if len(consumos_recurso) != len(limites_recursos):
                raise ValueError(f"Aresta ({u},{v}) tem {len(consumos_recurso)} consumos, mas {len(limites_recursos)} limites definidos.")
            self.adj[u].append({'para': v, 'custo': custo, 'consumos_recurso': consumos_recurso})
            if custo < 0:
                self.contem_custos_negativos = True

        self.limites_recursos = list(limites_recursos)
        self.num_recursos = len(limites_recursos)
        self.rotulos = [[] for _ in range(num_nos)]
        self.melhor_custo_solucao = INF
        self.melhor_caminho_solucao = []
        self.max_nos_no_caminho = self.num_nos


    def _eh_dominado(self, no, custo, recursos_consumidos, num_arestas_atuais, caminho_atual_set):
        indices_rotulos_dominados_pelo_novo = []
        rotulos_a_manter = []

        for rot_custo, rot_recursos, rot_caminho, rot_num_arestas in self.rotulos[no]:
            novo_eh_melhor_ou_igual_em_recursos = all(
                rc_novo <= rc_rot for rc_novo, rc_rot in zip(recursos_consumidos, rot_recursos)
            )
            if custo <= rot_custo and novo_eh_melhor_ou_igual_em_recursos:
                if (custo < rot_custo or
                    any(rc_novo < rc_rot for rc_novo, rc_rot in zip(recursos_consumidos, rot_recursos)) or
                    (custo == rot_custo and all(rc_n == rc_r for rc_n,rc_r in zip(recursos_consumidos, rot_recursos)) and num_arestas_atuais < rot_num_arestas)):
                    indices_rotulos_dominados_pelo_novo.append( (rot_custo, rot_recursos, rot_caminho, rot_num_arestas) )
                    continue

            existente_eh_melhor_ou_igual_em_recursos = all(
                rc_rot <= rc_novo for rc_rot, rc_novo in zip(rot_recursos, recursos_consumidos)
            )
            if rot_custo <= custo and existente_eh_melhor_ou_igual_em_recursos:
                 if (rot_custo < custo or
                    any(rc_rot < rc_novo for rc_rot, rc_novo in zip(rot_recursos, recursos_consumidos)) or
                    (rot_custo == custo and all(rc_r == rc_n for rc_r,rc_n in zip(rot_recursos, recursos_consumidos)) and rot_num_arestas < num_arestas_atuais )):
                    return True

            rotulos_a_manter.append((rot_custo, rot_recursos, rot_caminho, rot_num_arestas))

        self.rotulos[no] = rotulos_a_manter
        return False


    def _podar_por_recursos(self, recursos_consumidos):
        for i in range(self.num_recursos):
            if recursos_consumidos[i] > self.limites_recursos[i] + 1e-9:
                return True
        return False

    def resolver(self, no_inicial, no_final):
        self.melhor_custo_solucao = INF
        self.melhor_caminho_solucao = []
        for i in range(self.num_nos): self.rotulos[i] = []

        recursos_iniciais = [0] * self.num_recursos
        pulso_inicial = Pulso(no_inicial, 0, recursos_iniciais, [no_inicial], 0)
        pilha_pulsos = [pulso_inicial]

        while pilha_pulsos:
            pulso_atual = pilha_pulsos.pop()
            no_corrente = pulso_atual.no_atual
            custo_corrente = pulso_atual.custo
            recursos_correntes = pulso_atual.recursos_consumidos
            caminho_corrente_lista = pulso_atual.caminho
            num_arestas_corrente = pulso_atual.num_arestas

            if len(caminho_corrente_lista) > self.max_nos_no_caminho :
                 continue

            if custo_corrente >= self.melhor_custo_solucao:
                if not (custo_corrente == self.melhor_custo_solucao and no_corrente == no_final):
                    continue

            if self._podar_por_recursos(recursos_correntes): continue

            if self._eh_dominado(no_corrente, custo_corrente, recursos_correntes, num_arestas_corrente, set(caminho_corrente_lista)): continue

            self.rotulos[no_corrente].append((custo_corrente, list(recursos_correntes), list(caminho_corrente_lista), num_arestas_corrente))

            if no_corrente == no_final:
                if custo_corrente < self.melhor_custo_solucao:
                    self.melhor_custo_solucao = custo_corrente
                    self.melhor_caminho_solucao = list(caminho_corrente_lista)
                elif custo_corrente == self.melhor_custo_solucao and len(caminho_corrente_lista) < len(self.melhor_caminho_solucao):
                    self.melhor_caminho_solucao = list(caminho_corrente_lista)
                continue

            for aresta in self.adj[no_corrente]:
                no_vizinho = aresta['para']

                if no_vizinho in caminho_corrente_lista:
                    continue

                novo_custo = custo_corrente + aresta['custo']
                novos_recursos = [recursos_correntes[i] + aresta['consumos_recurso'][i] for i in range(self.num_recursos)]
                nova_num_arestas = num_arestas_corrente + 1

                if novo_custo < self.melhor_custo_solucao or (novo_custo == self.melhor_custo_solucao and no_vizinho == no_final):
                    if not self._podar_por_recursos(novos_recursos):
                        novo_caminho_lista = list(caminho_corrente_lista) + [no_vizinho]
                        if len(novo_caminho_lista) <= self.max_nos_no_caminho:
                            novo_pulso = Pulso(no_vizinho, novo_custo, novos_recursos, novo_caminho_lista, nova_num_arestas)
                            pilha_pulsos.append(novo_pulso)

        if self.melhor_custo_solucao == INF:
            return None, []
        return self.melhor_custo_solucao, self.melhor_caminho_solucao

# ==============================================================================
# CLASSE ESTADOASTAR (Caminhos Elementares)
# ==============================================================================
class EstadoAStar:
    def __init__(self, no_atual, g_custo, h_custo, recursos_consumidos, caminho_lista_nos):
        self.no_atual = no_atual
        self.g_custo = g_custo
        self.h_custo = h_custo
        self.f_custo = self.g_custo + self.h_custo
        self.recursos_consumidos = list(recursos_consumidos)
        self.caminho = list(caminho_lista_nos)

    def __lt__(self, other):
        if self.f_custo != other.f_custo: return self.f_custo < other.f_custo
        if self.g_custo != other.g_custo: return self.g_custo < other.g_custo
        return self.no_atual < other.no_atual

    def _get_estado_chave(self):
        # Para caminhos elementares, a chave do estado para g_custos pode precisar
        # incluir o conjunto de nós visitados (frozenset(self.caminho[:-1])) para distinguir
        # estados que chegam ao self.no_atual por caminhos diferentes.
        # No entanto, isso aumenta drasticamente o número de estados.
        # Simplificação: a chave ainda é (nó, tupla_recursos). A poda de ciclo lida com elementaridade.
        # Isso significa que um estado (nó, recursos) pode ser atualizado se um caminho elementar
        # diferente e melhor o alcançar.
        return (self.no_atual, tuple(self.recursos_consumidos), frozenset(self.caminho[:-1]))


    def __hash__(self):
        return hash(self._get_estado_chave())

    def __eq__(self, other):
        if not isinstance(other, EstadoAStar): return NotImplemented
        return self._get_estado_chave() == other._get_estado_chave()


# ==============================================================================
# SOLUCIONADOR RCSPP COM ALGORITMO A* (Caminhos Elementares)
# ==============================================================================
class SolucionadorRCSPPAStar:
    def __init__(self, num_nos, arestas, limites_recursos):
        self.num_nos = num_nos
        self.adj = [[] for _ in range(num_nos)]
        self.contem_custos_negativos = False
        for u, v, custo, consumos_recurso in arestas:
            if not (0 <= u < num_nos and 0 <= v < num_nos):
                raise ValueError(f"Nó inválido na aresta ({u}, {v}).")
            if len(consumos_recurso) != len(limites_recursos):
                 raise ValueError(f"Aresta ({u},{v}) tem consumos inconsistentes.")
            self.adj[u].append({'para': v, 'custo': custo, 'consumos_recurso': consumos_recurso})
            if custo < 0: self.contem_custos_negativos = True
        self.limites_recursos = list(limites_recursos)
        self.num_recursos = len(limites_recursos)
        self.melhor_solucao_final = None
        self.max_nos_no_caminho = self.num_nos


    def _calcular_heuristica(self, no_corrente, no_destino):
        return 0

    def _podar_por_recursos(self, recursos_consumidos):
        for i in range(self.num_recursos):
            if recursos_consumidos[i] > self.limites_recursos[i] + 1e-9: return True
        return False

    def resolver(self, no_inicial, no_final):
        self.melhor_solucao_final = None
        lista_aberta = []

        recursos_iniciais = [0] * self.num_recursos
        h_inicial = self._calcular_heuristica(no_inicial, no_final)
        estado_inicial = EstadoAStar(no_inicial, 0, h_inicial, recursos_iniciais, [no_inicial])
        heapq.heappush(lista_aberta, estado_inicial)

        g_custos_por_estado = {}
        g_custos_por_estado[estado_inicial._get_estado_chave()] = 0

        contador_atualizacoes_estado = {}
        # Para caminhos elementares, um estado (nó, recursos, caminho_visitado) não deve ser expandido mais de uma vez.
        # O limite de atualizações é mais para SPFA não elementar.
        # Aqui, se um estado (com seu caminho específico) é retirado, é o ótimo para *aquele* caminho.

        while lista_aberta:
            estado_atual = heapq.heappop(lista_aberta)
            no_corrente = estado_atual.no_atual
            g_custo_corrente = estado_atual.g_custo
            recursos_correntes = estado_atual.recursos_consumidos
            caminho_corrente_lista = estado_atual.caminho

            if len(caminho_corrente_lista) > self.max_nos_no_caminho:
                continue

            chave_estado_corrente_dict = estado_atual._get_estado_chave()

            if self.melhor_solucao_final and g_custo_corrente >= self.melhor_solucao_final.g_custo:
                 if not (no_corrente == no_final and g_custo_corrente == self.melhor_solucao_final.g_custo):
                    continue

            # Se já encontramos um caminho melhor para ESTE ESTADO ESPECÍFICO (incluindo o caminho visitado)
            if g_custo_corrente > g_custos_por_estado.get(chave_estado_corrente_dict, INF):
                continue

            if no_corrente == no_final:
                if self.melhor_solucao_final is None or g_custo_corrente < self.melhor_solucao_final.g_custo:
                    self.melhor_solucao_final = estado_atual
                elif g_custo_corrente == self.melhor_solucao_final.g_custo and len(caminho_corrente_lista) < len(self.melhor_solucao_final.caminho):
                    self.melhor_solucao_final = estado_atual
                continue

            for aresta in self.adj[no_corrente]:
                no_vizinho = aresta['para']

                if no_vizinho in caminho_corrente_lista: # Garante caminho elementar
                    continue

                custo_aresta = aresta['custo']
                consumos_aresta = aresta['consumos_recurso']
                novo_g_custo = g_custo_corrente + custo_aresta
                novos_recursos = [recursos_correntes[i] + consumos_aresta[i] for i in range(self.num_recursos)]

                if self._podar_por_recursos(novos_recursos): continue

                h_vizinho = self._calcular_heuristica(no_vizinho, no_final)
                if self.melhor_solucao_final and (novo_g_custo + h_vizinho) >= self.melhor_solucao_final.g_custo:
                    if not (no_vizinho == no_final and (novo_g_custo + h_vizinho) == self.melhor_solucao_final.g_custo):
                         continue

                novo_caminho_lista = list(caminho_corrente_lista) + [no_vizinho]
                if len(novo_caminho_lista) > self.max_nos_no_caminho: continue

                novo_estado_obj = EstadoAStar(no_vizinho, novo_g_custo, h_vizinho, novos_recursos, caminho_corrente_lista)
                chave_estado_vizinho_dict = novo_estado_obj._get_estado_chave()

                if novo_g_custo < g_custos_por_estado.get(chave_estado_vizinho_dict, INF):
                    g_custos_por_estado[chave_estado_vizinho_dict] = novo_g_custo
                    heapq.heappush(lista_aberta, novo_estado_obj)

        if self.melhor_solucao_final:
            return self.melhor_solucao_final.g_custo, self.melhor_solucao_final.caminho
        return None, []

# ==============================================================================
# CLASSE ESTADOSSR (Caminhos Elementares)
# ==============================================================================
class EstadoSSR:
    def __init__(self, no_atual, g_custo, recursos_consumidos_reais, buckets_recursos, caminho_lista_nos):
        self.no_atual = no_atual
        self.g_custo = g_custo
        self.recursos_consumidos_reais = list(recursos_consumidos_reais)
        self.buckets_recursos = tuple(buckets_recursos)
        self.caminho = list(caminho_lista_nos)

    def __lt__(self, other):
        if self.g_custo != other.g_custo: return self.g_custo < other.g_custo
        return self.no_atual < other.no_atual

    def _get_estado_chave(self):
        # Para SSR com caminhos elementares, a chave do estado na g_custos_ssr
        # também deveria, idealmente, incluir o conjunto de nós visitados.
        # Simplificação: (nó, tupla_buckets). A poda de ciclo garante elementaridade.
        return (self.no_atual, self.buckets_recursos, frozenset(self.caminho[:-1]))


    def __hash__(self):
        return hash(self._get_estado_chave())

    def __eq__(self, other):
        if not isinstance(other, EstadoSSR): return NotImplemented
        return self._get_estado_chave() == other._get_estado_chave()

# ==============================================================================
# SOLUCIONADOR RCSPP COM ALGORITMO SSR-INSPIRED (Caminhos Elementares)
# ==============================================================================
class SolucionadorRCSPP_SSRInspired:
    def __init__(self, num_nos, arestas, limites_recursos, num_buckets_por_recurso=5):
        self.num_nos = num_nos
        self.adj = [[] for _ in range(num_nos)]
        self.contem_custos_negativos = False
        for u, v, custo, consumos_recurso in arestas:
            if not (0 <= u < num_nos and 0 <= v < num_nos):
                raise ValueError(f"Nó inválido na aresta ({u}, {v}).")
            if len(consumos_recurso) != len(limites_recursos):
                 raise ValueError(f"Aresta ({u},{v}) tem consumos inconsistentes.")
            self.adj[u].append({'para': v, 'custo': custo, 'consumos_recurso': consumos_recurso})
            if custo < 0: self.contem_custos_negativos = True

        self.limites_recursos_reais = list(limites_recursos)
        self.num_recursos = len(limites_recursos)
        self.num_buckets_por_recurso = num_buckets_por_recurso
        self.tamanho_bucket = [(lim / num_buckets_por_recurso) if num_buckets_por_recurso > 0 and lim > 0 else INF
                               for lim in self.limites_recursos_reais]
        for i in range(self.num_recursos):
            if self.limites_recursos_reais[i] > 0 and self.tamanho_bucket[i] == 0 and num_buckets_por_recurso > 0 :
                 self.tamanho_bucket[i] = self.limites_recursos_reais[i] / num_buckets_por_recurso
            elif self.limites_recursos_reais[i] == 0 :
                 self.tamanho_bucket[i] = 1e-9

        self.melhor_solucao_final_ssr = None
        self.max_nos_no_caminho = self.num_nos

    def _get_bucket_indices(self, recursos_consumidos_reais):
        indices = []
        for i in range(self.num_recursos):
            if self.tamanho_bucket[i] == INF or self.num_buckets_por_recurso == 0:
                bucket_idx = 0
            elif self.tamanho_bucket[i] < 1e-8 :
                bucket_idx = self.num_buckets_por_recurso if recursos_consumidos_reais[i] > 1e-9 else 0
            else:
                bucket_idx = int(recursos_consumidos_reais[i] / self.tamanho_bucket[i])
            indices.append(min(bucket_idx, self.num_buckets_por_recurso))
        return tuple(indices)

    def _podar_por_recursos_reais(self, recursos_consumidos_reais):
        for i in range(self.num_recursos):
            if recursos_consumidos_reais[i] > self.limites_recursos_reais[i] + 1e-9:
                return True
        return False

    def resolver(self, no_inicial, no_final):
        self.melhor_solucao_final_ssr = None
        lista_aberta_ssr = []

        recursos_reais_iniciais = [0] * self.num_recursos
        buckets_iniciais = self._get_bucket_indices(recursos_reais_iniciais)
        estado_inicial_ssr = EstadoSSR(no_inicial, 0, recursos_reais_iniciais, buckets_iniciais, [no_inicial])
        heapq.heappush(lista_aberta_ssr, estado_inicial_ssr)

        g_custos_ssr = {}
        g_custos_ssr[estado_inicial_ssr._get_estado_chave()] = 0

        while lista_aberta_ssr:
            estado_atual_ssr = heapq.heappop(lista_aberta_ssr)
            no_corrente = estado_atual_ssr.no_atual
            g_custo_corrente_real = estado_atual_ssr.g_custo
            recursos_correntes_reais = estado_atual_ssr.recursos_consumidos_reais
            caminho_corrente_lista = estado_atual_ssr.caminho

            if len(caminho_corrente_lista) > self.max_nos_no_caminho: continue

            chave_estado_discreto_corrente_dict = estado_atual_ssr._get_estado_chave()

            if self.melhor_solucao_final_ssr and g_custo_corrente_real >= self.melhor_solucao_final_ssr.g_custo:
                if not (no_corrente == no_final and g_custo_corrente_real == self.melhor_solucao_final_ssr.g_custo):
                    continue

            if g_custo_corrente_real > g_custos_ssr.get(chave_estado_discreto_corrente_dict, INF):
                continue

            if no_corrente == no_final:
                if not self._podar_por_recursos_reais(recursos_correntes_reais):
                    if self.melhor_solucao_final_ssr is None or g_custo_corrente_real < self.melhor_solucao_final_ssr.g_custo:
                        self.melhor_solucao_final_ssr = estado_atual_ssr
                    elif g_custo_corrente_real == self.melhor_solucao_final_ssr.g_custo and len(caminho_corrente_lista) < len(self.melhor_solucao_final_ssr.caminho):
                        self.melhor_solucao_final_ssr = estado_atual_ssr
                continue

            for aresta in self.adj[no_corrente]:
                no_vizinho, custo_aresta, consumos_aresta = aresta['para'], aresta['custo'], aresta['consumos_recurso']

                if no_vizinho in caminho_corrente_lista: # Garante caminho elementar
                    continue

                novo_g_custo_real = g_custo_corrente_real + custo_aresta
                novos_recursos_reais = [recursos_correntes_reais[i] + consumos_aresta[i] for i in range(self.num_recursos)]

                if self._podar_por_recursos_reais(novos_recursos_reais): continue

                novo_caminho_lista = list(caminho_corrente_lista) + [no_vizinho]
                if len(novo_caminho_lista) > self.max_nos_no_caminho: continue

                novos_buckets = self._get_bucket_indices(novos_recursos_reais)

                if self.melhor_solucao_final_ssr and novo_g_custo_real >= self.melhor_solucao_final_ssr.g_custo:
                     if not (no_vizinho == no_final and novo_g_custo_real == self.melhor_solucao_final_ssr.g_custo):
                        continue

                novo_estado_ssr_obj = EstadoSSR(no_vizinho, novo_g_custo_real, novos_recursos_reais, novos_buckets, caminho_corrente_lista)
                chave_estado_discreto_vizinho_dict = novo_estado_ssr_obj._get_estado_chave()

                if novo_g_custo_real < g_custos_ssr.get(chave_estado_discreto_vizinho_dict, INF):
                    g_custos_ssr[chave_estado_discreto_vizinho_dict] = novo_g_custo_real
                    heapq.heappush(lista_aberta_ssr, novo_estado_ssr_obj)

        if self.melhor_solucao_final_ssr:
            if not self._podar_por_recursos_reais(self.melhor_solucao_final_ssr.recursos_consumidos_reais):
                 return self.melhor_solucao_final_ssr.g_custo, self.melhor_solucao_final_ssr.caminho
        return None, []

# ==============================================================================
# SOLUCIONADOR RCSPP COM ALGORITMO ADVANCEDPULSE (Caminhos Elementares)
# ==============================================================================
class SolucionadorRCSPP_AdvancedPulse(SolucionadorRCSPPPulse):
    def __init__(self, num_nos, arestas, limites_recursos):
        super().__init__(num_nos, arestas, limites_recursos)
        self.heuristics_to_dest = {i: 0 for i in range(num_nos)}

    def _precompute_heuristics(self, no_alvo):
        if self.contem_custos_negativos:
            self.heuristics_to_dest = {i: 0 for i in range(self.num_nos)}
            return

        dist_h = {i: INF for i in range(self.num_nos)}
        dist_h[no_alvo] = 0
        adj_reverso = [[] for _ in range(self.num_nos)]
        for u_orig in range(self.num_nos):
            for aresta in self.adj[u_orig]:
                v_orig, custo = aresta['para'], aresta['custo']
                adj_reverso[v_orig].append({'para': u_orig, 'custo': custo})

        pq = [(0, no_alvo)]
        while pq:
            h_atual, no_u = heapq.heappop(pq)
            if h_atual > dist_h[no_u]: continue
            for aresta_rev in adj_reverso[no_u]:
                no_v, custo_aresta = aresta_rev['para'], aresta_rev['custo']
                if dist_h[no_u] + custo_aresta < dist_h[no_v]:
                    dist_h[no_v] = dist_h[no_u] + custo_aresta
                    heapq.heappush(pq, (dist_h[no_v], no_v))
        self.heuristics_to_dest = dist_h


    def resolver(self, no_inicial, no_final):
        self._precompute_heuristics(no_final)

        self.melhor_custo_solucao = INF
        self.melhor_caminho_solucao = []
        for i in range(self.num_nos): self.rotulos[i] = []

        recursos_iniciais = [0] * self.num_recursos
        h_inicial_val = self.heuristics_to_dest.get(no_inicial, 0)

        pulso_inicial = Pulso(no_inicial, 0, recursos_iniciais, [no_inicial], 0)
        fila_prioridade_pulsos = [(0 + h_inicial_val, 0, pulso_inicial)]


        while fila_prioridade_pulsos:
            f_estimado, custo_corrente_g, pulso_atual = heapq.heappop(fila_prioridade_pulsos)

            no_corrente = pulso_atual.no_atual
            recursos_correntes = pulso_atual.recursos_consumidos
            caminho_corrente_lista = pulso_atual.caminho
            num_arestas_corrente = pulso_atual.num_arestas

            if len(caminho_corrente_lista) > self.max_nos_no_caminho: continue

            if f_estimado >= self.melhor_custo_solucao:
                 if not (f_estimado == self.melhor_custo_solucao and no_corrente == no_final):
                    continue
            if custo_corrente_g >= self.melhor_custo_solucao:
                if not (custo_corrente_g == self.melhor_custo_solucao and no_corrente == no_final):
                    continue

            if self._podar_por_recursos(recursos_correntes): continue
            if self._eh_dominado(no_corrente, custo_corrente_g, recursos_correntes, num_arestas_corrente, set(caminho_corrente_lista)): continue

            self.rotulos[no_corrente].append((custo_corrente_g, list(recursos_correntes), list(caminho_corrente_lista), num_arestas_corrente))

            if no_corrente == no_final:
                if custo_corrente_g < self.melhor_custo_solucao:
                    self.melhor_custo_solucao = custo_corrente_g
                    self.melhor_caminho_solucao = list(caminho_corrente_lista)
                elif custo_corrente_g == self.melhor_custo_solucao and len(caminho_corrente_lista) < len(self.melhor_caminho_solucao):
                    self.melhor_caminho_solucao = list(caminho_corrente_lista)
                continue

            for aresta in self.adj[no_corrente]:
                no_vizinho = aresta['para']

                if no_vizinho in caminho_corrente_lista: # Garante caminho elementar
                    continue

                novo_custo_g = custo_corrente_g + aresta['custo']
                h_vizinho_val = self.heuristics_to_dest.get(no_vizinho, 0)
                novo_f_estimado = novo_custo_g + h_vizinho_val

                if novo_f_estimado >= self.melhor_custo_solucao:
                    if not (novo_f_estimado == self.melhor_custo_solucao and no_vizinho == no_final):
                        continue
                if novo_custo_g >= self.melhor_custo_solucao:
                    if not (novo_custo_g == self.melhor_custo_solucao and no_vizinho == no_final):
                        continue

                novos_recursos = [recursos_correntes[i] + aresta['consumos_recurso'][i] for i in range(self.num_recursos)]
                nova_num_arestas = num_arestas_corrente + 1

                if not self._podar_por_recursos(novos_recursos):
                    novo_caminho_lista = list(caminho_corrente_lista) + [no_vizinho]
                    if len(novo_caminho_lista) <= self.max_nos_no_caminho:
                        novo_pulso = Pulso(no_vizinho, novo_custo_g, novos_recursos, novo_caminho_lista, nova_num_arestas)
                        heapq.heappush(fila_prioridade_pulsos, (novo_f_estimado, novo_custo_g, novo_pulso))

        if self.melhor_custo_solucao == INF:
            return None, []
        return self.melhor_custo_solucao, self.melhor_caminho_solucao

# ==============================================================================
# FUNÇÃO DE PARSING DE INSTÂNCIA
# ==============================================================================
def parse_instancia_benchmark(filepath):
    arestas, num_nos, limites_recursos, no_inicial, no_final, num_tipos_recurso = [], 0, [], -1, -1, 0
    try:
        with open(filepath, 'r') as f:
            linhas = [l.strip() for l in f if l.strip()]
            if not linhas: raise ValueError("Arquivo de instância vazio.")

            parts_h = linhas[0].split()
            if len(parts_h) < 5: raise ValueError("Cabeçalho incompleto.")
            num_nos, num_tipos_recurso = int(parts_h[0]), int(parts_h[2])
            no_inicial, no_final = int(parts_h[3]) - 1, int(parts_h[4]) - 1

            parts_l = linhas[1].split()
            if len(parts_l) != num_tipos_recurso: raise ValueError("Limites de recursos inconsistentes.")
            limites_recursos = [float(r) for r in parts_l]

            for i in range(2, len(linhas)):
                parts_a = linhas[i].split()
                if len(parts_a) < 3 + num_tipos_recurso: continue
                u, v = int(parts_a[0]) - 1, int(parts_a[1]) - 1
                custo = float(parts_a[2])
                consumos = [float(parts_a[j]) for j in range(3, 3 + num_tipos_recurso)]
                if len(consumos) != num_tipos_recurso: continue
                arestas.append((u, v, custo, consumos))
    except FileNotFoundError: print(f"Erro: Arquivo não encontrado: {filepath}"); return None
    except ValueError as ve: print(f"Erro de valor ao parsear {filepath}: {ve}"); return None
    except Exception as e: print(f"Erro inesperado ao parsear {filepath}: {e}"); return None
    return num_nos, arestas, limites_recursos, no_inicial, no_final

# ==============================================================================
# GERADOR DE INSTÂNCIA (com custos potencialmente negativos)
# ==============================================================================
def gerar_instancia_rccsp(filepath, num_nos, densidade=0.3, num_arestas_alvo_fallback=None, # Adicionado densidade
                           num_tipos_recurso=2, no_origem_arq=1, no_destino_arq=None,
                           max_custo_aresta=30.0, min_custo_aresta = -10.0,
                           prob_custo_negativo=0.1,
                           max_consumo_recurso=10.0):
    if no_destino_arq is None:
        no_destino_arq = num_nos # Padrão para o último nó
    if not (1 <= no_origem_arq <= num_nos and 1 <= no_destino_arq <= num_nos):
        raise ValueError("Nó de origem ou destino fora do intervalo.")

    num_arestas_a_gerar = 0
    if densidade is not None and 0 < densidade <= 1:
        num_arestas_a_gerar = int(densidade * num_nos * (num_nos - 1))
        if num_nos <= 1: num_arestas_a_gerar = 0 # Evita N*(N-1) negativo ou zero para N pequeno
    elif num_arestas_alvo_fallback is not None:
        num_arestas_a_gerar = num_arestas_alvo_fallback
    else: # Fallback para uma densidade padrão se nenhum for fornecido
        num_arestas_a_gerar = int(0.3 * num_nos * (num_nos - 1))
        if num_nos <=1: num_arestas_a_gerar = 0

    # Garante um mínimo de arestas se o cálculo de densidade for muito baixo para grafos pequenos
    if num_nos > 1 and num_arestas_a_gerar < num_nos -1 : # Pelo menos para um caminho simples
        num_arestas_a_gerar = num_nos -1
    if num_nos ==1 : num_arestas_a_gerar = 0


    output_lines = []
    # Limites de recursos um pouco mais apertados para tornar o problema mais interessante
    limites_recursos = [round(num_nos * max_consumo_recurso * random.uniform(0.15, 0.25), 1) for _ in range(num_tipos_recurso)]
    # Garantir que limites não sejam zero se o consumo máximo for > 0
    for i in range(len(limites_recursos)):
        if limites_recursos[i] < 1.0 and max_consumo_recurso > 0:
            limites_recursos[i] = round(max_consumo_recurso * (num_nos / 4),1) # Um valor mínimo
        if limites_recursos[i] == 0 and max_consumo_recurso > 0: # Se ainda for zero
             limites_recursos[i] = max_consumo_recurso


    arestas_geradas_str, set_arestas_existentes = [], set()

    # Tenta criar um caminho base com custos positivos (se possível)
    # para aumentar a chance de uma solução viável existir como referência.
    if no_origem_arq != no_destino_arq and len(arestas_geradas_str) < num_arestas_a_gerar:
        # Gerar um caminho sequencial simples se possível
        path_nodes_temp = []
        if no_origem_arq < no_destino_arq:
            path_nodes_temp = list(range(no_origem_arq -1, no_destino_arq))
        elif no_origem_arq > no_destino_arq: # Caminho reverso
            path_nodes_temp = list(range(no_origem_arq -1, no_destino_arq -2, -1))


        for i_path in range(len(path_nodes_temp) - 1):
            u, v = path_nodes_temp[i_path], path_nodes_temp[i_path+1]
            if (u,v) not in set_arestas_existentes and len(arestas_geradas_str) < num_arestas_a_gerar:
                custo_base = round(random.uniform(1.0, max_custo_aresta / 2), 1)
                consumos = [round(random.uniform(0.1, max_consumo_recurso / 2), 1) for _ in range(num_tipos_recurso)]
                arestas_geradas_str.append(f"{u+1} {v+1} {custo_base} {' '.join(map(str, consumos))}")
                set_arestas_existentes.add((u,v))

    # Adiciona arestas aleatórias até atingir o alvo
    max_possible_edges = num_nos * (num_nos - 1)
    while len(arestas_geradas_str) < num_arestas_a_gerar and len(set_arestas_existentes) < max_possible_edges :
        u, v = random.randint(0, num_nos - 1), random.randint(0, num_nos - 1)
        if u == v or (u,v) in set_arestas_existentes: continue

        custo_aresta_rand = 0
        if random.random() < prob_custo_negativo and min_custo_aresta < 0:
            custo_aresta_rand = round(random.uniform(min_custo_aresta, -0.1), 1)
        else:
            custo_aresta_rand = round(random.uniform(0.1, max_custo_aresta), 1)

        consumos = [round(random.uniform(0.1, max_consumo_recurso), 1) for _ in range(num_tipos_recurso)]
        arestas_geradas_str.append(f"{u+1} {v+1} {custo_aresta_rand} {' '.join(map(str, consumos))}")
        set_arestas_existentes.add((u,v))
        # Safety break se algo der errado e o alvo for muito alto
        if len(arestas_geradas_str) > num_arestas_a_gerar * 1.2 and len(arestas_geradas_str) > max_possible_edges: break


    header_line = f"{num_nos} {len(arestas_geradas_str)} {num_tipos_recurso} {no_origem_arq} {no_destino_arq}"
    output_lines.extend([header_line, " ".join(map(str, limites_recursos))])
    output_lines.extend(arestas_geradas_str)
    try:
        with open(filepath, "w") as f: f.write("\n".join(output_lines))
        print(f"Instância gerada (densidade, custos pot. negativos, elementar): {filepath}, {len(arestas_geradas_str)} arestas.")
    except IOError as e: print(f"Erro ao salvar instância: {e}")

# ==============================================================================
# BLOCO PRINCIPAL DE EXECUÇÃO
# ==============================================================================
if __name__ == '__main__':
    nome_arquivo_instancia = "minha_instancia_densa_elem_neg.txt"
    NUM_BUCKETS_SSR = 5
    # Parâmetros para gerar a instância
    NUM_NOS_INST = 1000 # Reduzido para testes rápidos de grafos densos
    DENSIDADE_INST = 0.6 # Ex: 60% de densidade. Para N=15, N*(N-1) = 15*14 = 210. 0.6*210 = 126 arestas.
    # Se quiser usar num_arestas_alvo_fallback, defina densidade=None

    print(f"Gerando nova instância (densidade={DENSIDADE_INST}, elementar, custos pot. negativos): {nome_arquivo_instancia}")
    gerar_instancia_rccsp(
        filepath=nome_arquivo_instancia,
        num_nos=NUM_NOS_INST,
        densidade=DENSIDADE_INST,
        # num_arestas_alvo_fallback=50, # Use este se densidade=None
        num_tipos_recurso=3,
        no_origem_arq=1,
        no_destino_arq=NUM_NOS_INST, # Destino é o último nó
        max_custo_aresta=10.0, # Custos menores para grafos densos
        min_custo_aresta=-3.0,
        prob_custo_negativo=0.3, # Maior probabilidade de negativos
        max_consumo_recurso=5.0 # Consumos menores
    )
    print("-" * 70)

    print(f"Carregando instância: {nome_arquivo_instancia}")
    dados_parseados = parse_instancia_benchmark(nome_arquivo_instancia)
    resultados_algoritmos = []

    if dados_parseados:
        num_nos, arestas, limites_recursos, no_inicial, no_final = dados_parseados
        print(f"Instância: {num_nos} nós, {len(arestas)} arestas. Origem: {no_inicial+1}, Destino: {no_final+1}")
        print(f"Limites de Recursos: {limites_recursos}")
        instancia_tem_negativos = any(aresta[2] < 0 for aresta in arestas)
        print(f"Instância contém custos negativos: {instancia_tem_negativos}")


        algoritmos_para_testar = [
            ("Pulse (Elem.)", SolucionadorRCSPPPulse),
            ("A* (SPFA-like Elem., h=0)", SolucionadorRCSPPAStar),
            (f"SSR-Inspired Elem. ({NUM_BUCKETS_SSR}b)", SolucionadorRCSPP_SSRInspired),
            ("AdvPulse (Elem., Bound, h adapt.)", SolucionadorRCSPP_AdvancedPulse)
        ]

        for nome_alg, classe_solucionador in algoritmos_para_testar:
            print(f"\n--- Executando {nome_alg} ---")
            try:
                if nome_alg.startswith("SSR-Inspired"):
                    solucionador = classe_solucionador(num_nos, arestas, limites_recursos, num_buckets_por_recurso=NUM_BUCKETS_SSR)
                else:
                    solucionador = classe_solucionador(num_nos, arestas, limites_recursos)

                tempo_inicio = time.time()
                custo, caminho = solucionador.resolver(no_inicial, no_final)
                tempo_fim = time.time()
                tempo_exec = tempo_fim - tempo_inicio

                if custo is not None:
                    print(f"{nome_alg} - Custo: {custo:.2f}, Nós: {len(caminho) if caminho else 'N/A'}, Tempo: {tempo_exec:.4f}s")
                    if caminho and len(caminho) != len(set(caminho)):
                        print(f"ALERTA {nome_alg}: Caminho encontrado NÃO é elementar: {[n+1 for n in caminho]}")

                    resultados_algoritmos.append({
                        "Algoritmo": nome_alg, "Custo": f"{custo:.2f}", "Nós no Caminho": len(caminho) if caminho else 0,
                        "Tempo (s)": f"{tempo_exec:.4f}", "Caminho": caminho if caminho else []
                    })
                else:
                    print(f"{nome_alg} - Nenhuma solução viável.")
                    resultados_algoritmos.append({
                        "Algoritmo": nome_alg, "Custo": "N/A", "Nós no Caminho": "N/A",
                        "Tempo (s)": f"{tempo_exec:.4f}", "Caminho": []
                    })
            except Exception as e:
                print(f"Erro no {nome_alg}: {e}")
                import traceback
                traceback.print_exc()
                resultados_algoritmos.append({
                    "Algoritmo": nome_alg, "Custo": "Erro", "Nós no Caminho": "Erro",
                    "Tempo (s)": "Erro", "Caminho": []
                })

        print("\n\n" + "="*70)
        print("TABELA COMPARATIVA DE RESULTADOS (CAMINHOS ELEMENTARES, GRAFO DENSO)")
        print("="*70)
        if resultados_algoritmos:
            header = ["Algoritmo", "Custo Solução", "Nós no Caminho", "Tempo (s)"]
            print(f"{header[0]:<35} | {header[1]:<15} | {header[2]:<15} | {header[3]:<10}")
            print("-" * 80)
            for res in resultados_algoritmos:
                print(f"{res['Algoritmo']:<35} | {str(res['Custo']):<15} | {str(res['Nós no Caminho']):<15} | {str(res['Tempo (s)']):<10}")

            print("\nVerificação de Consistência de Custo (entre soluções válidas):")
            solucoes_validas = [r for r in resultados_algoritmos if r["Custo"] not in ["N/A", "Erro"]]
            if len(solucoes_validas) > 1:
                custos_validos_float = []
                for r_val in solucoes_validas:
                    try: custos_validos_float.append(float(r_val["Custo"]))
                    except ValueError: pass

                if custos_validos_float:
                    primeiro_custo_float = custos_validos_float[0]
                    todos_mesmo_custo = all(abs(c - primeiro_custo_float) < 1e-5 for c in custos_validos_float)

                    if todos_mesmo_custo:
                        print(f"Todos os algoritmos que encontraram solução chegaram a custos MUITO PRÓXIMOS: ~{primeiro_custo_float:.2f}")
                    else:
                        print("ALERTA: Algoritmos encontraram soluções com CUSTOS DIFERENTES!")
                        for r_val in solucoes_validas: print(f"  - {r_val['Algoritmo']}: {r_val['Custo']}")
                else: print("Nenhuma solução com custo numérico válido para comparar.")
            elif solucoes_validas: print("Apenas um algoritmo encontrou uma solução válida com custo numérico.")
        else: print("Nenhum resultado para comparar.")
        print("="*70)
    else:
        print(f"Falha ao carregar a instância. Algoritmos não executados.")

Gerando nova instância (densidade=0.6, elementar, custos pot. negativos): minha_instancia_densa_elem_neg.txt
Instância gerada (densidade, custos pot. negativos, elementar): minha_instancia_densa_elem_neg.txt, 599400 arestas.
----------------------------------------------------------------------
Carregando instância: minha_instancia_densa_elem_neg.txt
Instância: 1000 nós, 599400 arestas. Origem: 1, Destino: 1000
Limites de Recursos: [1234.8, 836.6, 1175.0]
Instância contém custos negativos: True

--- Executando Pulse (Elem.) ---
Pulse (Elem.) - Custo: -20.50, Nós: 336, Tempo: 495.4662s

--- Executando A* (SPFA-like Elem., h=0) ---
A* (SPFA-like Elem., h=0) - Custo: -655.00, Nós: 1, Tempo: 4.0381s

--- Executando SSR-Inspired Elem. (5b) ---
SSR-Inspired Elem. (5b) - Custo: -655.00, Nós: 1, Tempo: 3.2600s

--- Executando AdvPulse (Elem., Bound, h adapt.) ---
AdvPulse (Elem., Bound, h adapt.) - Custo: -1018.10, Nós: 343, Tempo: 3.6158s


TABELA COMPARATIVA DE RESULTADOS (CAMINHOS ELEMENTAR